## Exploring the Numbers

In [ ]:
import pandas as pd

# Loading dataset
ind_df = pd.read_csv('data/indigenous_collection.csv', index_col='id')
print(f'Dataframe columns: \n{ind_df.columns}')

In [ ]:
# How many item do we have?
print(f'Amount of items: {len(ind_df)}')

# How many items per category?
print(f'Amount of items per category: \n{ind_df.groupby(["categoria"]).count()["url"]}')

In [ ]:
# How many items with images do we have?
print(f'Amount of items with images: {len(ind_df[ind_df["thumbnail"].notna()])}')

# How many items with images do we have per category?
print(f'Amount of items with image per category: \n{ind_df.groupby(["categoria"]).count()["thumbnail"]}')

In [ ]:
pd.set_option('display.max_rows', None)

# How many items do we have per community?
print(f'Amount of items per community: \n{ind_df.groupby(["povo"]).count()["url"]}')

In [ ]:
# How many items do we have per category per community?
print(f'Amount of items per category per community: \n{ind_df.groupby(["categoria", "povo"]).count()["url"]}')

In [ ]:
# How many items with image do we have per category per community?
print(f'Amount of items with image per category per community: \n{ind_df.groupby(["categoria", "povo"]).count()["thumbnail"]}')

In [ ]:
# Function to answer three basic questions about a column
# 1. How many unique values of the column do we have?
# 2. How many items have that column?
# 2. How many items do we have per unique value of the column?
def column_basic_info(column):
    n_unique_items = ind_df[column].nunique()
    print('Amount of unique ' + column + f': {n_unique_items}')
    print('Amount of items with an associated ' + column + f': {sum(ind_df[column].notna())}')
    
    if n_unique_items < len(ind_df)/4:
        print('Amount of items per ' + column + f': \n{ind_df.groupby([column]).count()["url"]}')
    
    print()

In [ ]:
# Exploring basic infor about other columns
avoid_columns = ['url', 'thumbnail', 'categoria', 'povo']
unique_columns = ['numero_do_item', 'descricao', 'image_path']
other_columns = [col for col in ind_df.columns if col not in avoid_columns]

for col in other_columns:
    column_basic_info(col)

In [ ]:
ind_df.columns

## Processing Columns